In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Tasiilaq,65.6145,-37.6368,14.00,67,75,3.53,GL,2021-01-31 22:35:25
1,1,Port Elizabeth,-33.9180,25.5701,71.60,73,40,21.85,ZA,2021-01-31 22:33:21
2,2,Bredasdorp,-34.5322,20.0403,63.00,83,99,1.99,ZA,2021-01-31 22:35:26
3,3,Hermanus,-34.4187,19.2345,63.00,83,91,1.99,ZA,2021-01-31 22:33:24
4,4,Ust-Nera,64.5667,143.2000,-29.24,74,99,7.40,RU,2021-01-31 22:35:26


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df  = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,Richards Bay,-28.7830,32.0377,77.52,89,95,8.01,ZA,2021-01-31 22:33:38
21,21,Salto De Agua,17.6167,-92.3667,86.00,62,40,5.08,MX,2021-01-31 22:35:29
24,24,Acarau,-2.8856,-40.1200,80.35,82,16,16.73,BR,2021-01-31 22:35:29
28,28,Navrongo,10.8956,-1.0921,78.19,16,16,8.32,GH,2021-01-31 22:35:30
35,35,Sawtell,-30.3833,153.1000,78.01,78,40,3.44,AU,2021-01-31 22:35:32
...,...,...,...,...,...,...,...,...,...,...
661,661,Cayenne,4.9333,-52.3333,80.60,78,75,6.91,GF,2021-01-31 22:37:39
668,668,Caxito,-8.5785,13.6643,80.60,78,40,10.36,AO,2021-01-31 22:37:41
669,669,Idah,7.1065,6.7342,79.43,46,32,5.35,NG,2021-01-31 22:37:41
673,673,Mackay,-21.1500,149.2000,80.60,74,40,10.36,AU,2021-01-31 22:33:26


In [9]:
clean_df = preferred_cities_df.dropna()

In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       182
City          182
Lat           182
Lng           182
Max Temp      182
Humidity      182
Cloudiness    182
Wind Speed    182
Country       182
Date          182
dtype: int64

In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       0
Date          0
dtype: int64

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
12,Richards Bay,ZA,77.52,-28.7830,32.0377,
21,Salto De Agua,MX,86.00,17.6167,-92.3667,
24,Acarau,BR,80.35,-2.8856,-40.1200,
28,Navrongo,GH,78.19,10.8956,-1.0921,
35,Sawtell,AU,78.01,-30.3833,153.1000,
42,Touros,BR,80.60,-5.1989,-35.4608,
43,Atuona,PF,78.98,-9.8000,-139.0333,
46,Conde,BR,80.60,-7.2597,-34.9075,
51,Hithadhoo,MV,82.20,-0.6000,73.0833,
52,Rikitea,PF,79.25,-23.1203,-134.9692,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


In [14]:
# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError): 
        print("No hotel found...skipping.")
        

No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
12,Richards Bay,ZA,77.52,-28.7830,32.0377,BON Hotel Waterfront Richards Bay
21,Salto De Agua,MX,86.00,17.6167,-92.3667,
24,Acarau,BR,80.35,-2.8856,-40.1200,castelo pizzaria
28,Navrongo,GH,78.19,10.8956,-1.0921,Azakka Hotel
35,Sawtell,AU,78.01,-30.3833,153.1000,Island Beach House Sawtell
...,...,...,...,...,...,...
661,Cayenne,GF,80.60,4.9333,-52.3333,Hôtel Le Dronmi
668,Caxito,AO,80.60,-8.5785,13.6643,
669,Idah,NG,79.43,7.1065,6.7342,Ubiga Hotel Idah
673,Mackay,AU,80.60,-21.1500,149.2000,International Lodge Motel


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""


In [18]:
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [19]:
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))